In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from tqdm import tqdm_notebook
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading files
df = pd.read_csv('radiant_pixels.csv')
sample_submission = pd.read_csv('SampleSubmission.csv')
train = df[df.crop_type != 0]
test = df[df.crop_type == 0]
target = train.crop_type
train.head()

,Field ID,crop_type,0_B01,0_B02,0_B03,0_B04,0_B05,0_B06,0_B07,0_B08,0_B8A,0_B09,0_B11,0_B12,0_CLM,1_B01,1_B02,1_B03,1_B04,1_B05,1_B06,1_B07,1_B08,1_B8A,1_B09,1_B11,1_B12,1_CLM,2_B01,2_B02,2_B03,2_B04,2_B05,2_B06,2_B07,2_B08,2_B8A,2_B09,2_B11,2_B12,...,21_CLM,22_B01,22_B02,22_B03,22_B04,22_B05,22_B06,22_B07,22_B08,22_B8A,22_B09,22_B11,22_B12,22_CLM,23_B01,23_B02,23_B03,23_B04,23_B05,23_B06,23_B07,23_B08,23_B8A,23_B09,23_B11,23_B12,23_CLM,24_B01,24_B02,24_B03,24_B04,24_B05,24_B06,24_B07,24_B08,24_B8A,24_B09,24_B11,24_B12,24_CLM
0,1,4,17.972797,21.714743,28.888264,35.199116,43.310113,51.914518,56.716495,61.865450,62.589646,63.789743,67.713135,45.579791,251.232395,12.392508,14.695611,22.883006,30.301640,39.662074,49.342428,55.085064,58.148971,61.867042,62.498923,72.701334,48.374711,0.0,10.714920,13.251994,21.380498,28.931238,37.393987,46.389936,51.411559,56.495257,58.435466,59.088039,72.932122,48.820370,...,0.0,13.942765,18.017363,27.907814,35.668650,45.651415,57.813135,63.208794,67.756109,68.415048,69.451206,84.713441,62.890466,0.0,14.528489,18.420048,28.003312,36.348280,46.310354,59.610096,65.381383,70.352074,71.209968,72.308199,88.385354,64.010981,0.0,11.829068,16.377974,27.102444,35.009711,44.464051,58.472074,64.281559,70.347299,69.858666,70.148344,87.061109,64.932540,0.0
1,2,7,57.678261,52.814348,59.959565,73.303043,81.438261,87.894783,94.265217,99.215217,102.787826,116.690870,96.460435,71.968696,252.450000,8.737826,20.316522,31.723043,46.443913,54.148696,61.853478,67.621304,73.173913,77.004783,64.651304,93.877826,65.813478,0.0,23.501739,24.061304,34.047391,46.228696,52.685217,59.615217,65.555217,72.614348,74.077826,86.732609,95.900870,67.320000,...,0.0,19.498696,20.703913,30.646957,45.367826,51.049565,54.406957,58.797391,65.296957,65.856522,71.753478,98.096087,78.769565,0.0,18.766957,19.843043,29.570870,44.636087,50.834348,54.062609,58.797391,63.919565,65.727391,69.859565,101.410435,80.448261,0.0,21.435652,22.296522,32.110435,45.927391,50.963478,54.493043,57.635217,64.263913,64.737391,68.180870,101.711739,84.580435,0.0
2,3,6,25.681465,29.684730,44.715424,63.017699,72.983869,79.479949,86.882044,93.826041,98.857481,99.869113,133.503663,102.065437,0.000000,9.861825,13.203393,22.841260,35.265887,43.029370,48.549447,55.445090,62.144769,68.589949,70.273458,109.919267,77.561028,0.0,13.820553,17.696568,26.936144,39.059190,45.436928,50.469640,56.623419,64.528149,67.995694,69.945154,108.828740,77.136015,...,0.0,23.139023,32.865964,48.134614,69.020051,77.792622,82.560656,87.900039,95.961285,96.872391,97.077262,115.557686,86.019293,0.0,23.416427,33.030116,49.284949,71.622301,81.350514,85.827147,91.365039,98.143612,100.460823,100.034537,123.538766,90.875129,0.0,26.002134,35.575103,51.581799,70.708650,78.400874,81.694087,86.362866,93.043458,93.674614,94.127622,124.671285,95.122712,0.0
3,4,8,30.585099,35.220397,42.799470,50.273642,55.249868,57.452781,60.219536,67.103642,65.602252,84.733510,79.914636,61.629139,252.450000,25.694106,32.689669,45.389205,60.449007,67.785497,71.430795,77.010199,82.156887,86.366026,86.733179,127.703444,110.257152,0.0,18.541192,27.962583,39.757351,53.623907,59.288543,63.569801,68.913179,76.170993,78.144437,75.200662,116.918344,99.072119,...,0.0,14.273046,21.248940,31.011258,43.002715,51.073510,57.819934,63.064967,69.824503,72.197881,69.011523,95.675960,74.944967,0.0,18.836225,28.618212,40.858808,57.944503,65.517020,72.217550,78.105099,87.264238,88.136225,82.851854,113.318940,86.103775,0.0,19.236159,28.428079,40.432649,55.171192,61.917616,69.378675,75.069536,84.064768,83.992649,78.727947,112.230596,87.828079,0.0
4,6,4,17.842500,18.978750,29.694375,37.361250,49.449375,65.671875,74.075625,79.936875,83.160000,83.328750,90.191250,59.760000,0.000000,15.198750,18.241875,28.856250,36.264375,47.812500,60.536250,67.680000,72.545625,76.179375,75.296250,89.251875,60.277500,0.0,13.545000,16.700625,27.039375,34.965000,45.208125,57.211875,63.534375,70.554375,72.022500,71.375625,86.445000,58.089375,...,0.0,14.551875,15.946875,27.

### Bands

In [ ]:
# Sentinel 2 band description

# Band 1 - Coastal aerosol	0.443	60
# Band 2 - Blue	0.490	10
# Band 3 - Green	0.560	10
# Band 4 - Red	0.665	10
# Band 5 - Vegetation Red Edge	0.705	20
# Band 6 - Vegetation Red Edge	0.740	20
# Band 7 - Vegetation Red Edge	0.783	20
# Band 8 - NIR	0.842	10
# Band 8A - Vegetation Red Edge	0.865	20
# Band 9 - Water vapour	0.945	60
# Band 10 - SWIR - Cirrus	1.375	60
# Band 11 - SWIR	1.610	20
# Band 12 - SWIR

In [ ]:
# Getting the bands
B01, B02, B03, B04, B05 = df.filter(regex='._B01'), df.filter(regex='._B02'), df.filter(regex='._B03'), df.filter(regex='._B04'), df.filter(regex='._B05')
B06, B07, B08, B8A, B09 = df.filter(regex='._B06'), df.filter(regex='._B07'), df.filter(regex='._B08'), df.filter(regex='._B8A'), df.filter(regex='._B09')
B11, B12 = df.filter(regex='._B11'), df.filter(regex='._B12')

In [ ]:
# Formulas for different vegetation indices
def aci(i): return B08.values[:, i] * (B04.values[:, i] + B03.values[:, i])
def avi (i): return (B08.values[:, i] * (1 - B04.values[:, i]) * (B08.values[:, i] - B04.values[:, i]))
def bai (i): return 1/((0.1 - B04.values[:, i]) ** 2 + (0.06 - B08.values[:, i]) ** 2)
def bgi (i): return B01.values[:, i] / B03.values[:, i]
def bri (i): return B01.values[:, i] / B05.values[:, i] 
def ccci (i): return ((B08.values[:, i] - B05.values[:, i]) / (B08.values[:, i] + B05.values[:, i])) / ((B08.values[:, i] - B04.values[:, i]) / (B08.values[:, i] + B04.values[:, i]))
def cm (i): return B11.values[:, i] / B12.values[:, i]
def dyi (i): return B03.values[:, i] - B02.values[:, i]
def evi (i): return 2.5*(B08.values[:, i] - B04.values[:, i]) / (B08.values[:, i] + 6*B04.values[:, i] - 7.5*B02.values[:, i] + 1)
def exg (i): return 2 * B03.values[:, i] - B04.values[:, i] - B02.values[:, i]
def fidet (i): return B12.values[:, i] / (B8A.values[:, i] * B09.values[:, i])
def mcari (i): return ((B05.values[:, i] - B04.values[:, i]) - 2 * (B05.values[:, i] - B03.values[:, i])) * (B05.values[:, i] / B04.values[:, i]) 
def mi (i): return (B8A.values[:, i] - B11.values[:, i]) / (B8A.values[:, i] + B11.values[:, i])
def mrendvi (i): return (B06.values[:, i] - B05.values[:, i]) / (B06.values[:, i] + B05.values[:, i] - 2 * B01.values[:, i])
def mresr (i): return (B06.values[:, i] - B01.values[:, i]) / (B05.values[:, i] - B01.values[:, i])
def mtci (i): return (B06.values[:, i] - B05.values[:, i])/(B05.values[:, i] - B04.values[:, i])
def mtvi (i): return [1.5*(1.2 * (a - b) - 2.5 * (c - b))* math.sqrt((2 * a + 1)**2-(6 * a - 5 * math.sqrt(c)) - 0.5) for a, b, c in zip(B08.values[:, i], B03.values[:, i], B04.values[:, i])]
def nbr (i): return (B08.values[:, i] - B11.values[:, i]) / (B08.values[:, i]+ B11.values[:, i])
def ndsi (i): return (B03.values[:, i] - B11.values[:, i]) / (B03.values[:, i] + B11.values[:, i])
def nli (i): return (B08.values[:, i] **2 - B04.values[:, i]) / (B08.values[:, i] **2 + B04.values[:, i])
def ndmi (i): return (B08.values[:, i] - B11.values[:, i])/(B08.values[:, i] + B11.values[:, i])
def pvi (i): return (B08.values[:, i] - 0.3 * B04.values[:, i] - 0.5) / (math.sqrt(1 + 0.3 * 2))
def s2rep (i): return 705 + 35 * ((((B07.values[:, i] + B04.values[:, i])/2) - B05.values[:, i])/(B06.values[:, i] - B05.values[:, i]))
def si (i): return ((1 - B02.values[:, i]) * (1 - B03.values[:, i]) * (1 - B04.values[:, i]))
def sipi (i): return (B08.values[:, i] - B02.values[:, i]) / (B08.values[:, i] - B04.values[:, i])
def sr3 (i): return B05.values[:, i] / B04.values[:, i]
def tcari (i): return 3 * ((B05.values[:, i] - B04.values[:, i]) - 0.2 * (B05.values[:, i] - B03.values[:, i]) * (B05.values[:, i] / B04.values[:, i]))
def tvi (i): return (120 * (B06.values[:, i] - B03.values[:, i]) - 200 * (B04.values[:, i] - B03.values[:, i])) / 2
def vdvi (i): return (2 * B03.values[:, i] - B04.values[:, i] - B02.values[:, i]) / (2 * B03.values[:, i] + B04.values[:, i]  + B02.values[:, i])

In [ ]:
# Helper functions
num_dates = 25
bin_labels = list(range(3))
indices_df = []

# Function to convert continuos vegetation into a category via pca
def cat_indices(indice):
  return pd.cut(PCA(1).fit_transform(pd.DataFrame(indice)).flatten(), len(bin_labels), labels = bin_labels)
# Calculating indices
def indice_calculator(func_name, indice_name):
  holder = []
  for i in range(num_dates):
    holder.append(func_name(i))
  indice_df = pd.DataFrame(np.array(holder).T, columns = [indice_name + '_date_' + str(i) for i in range(1, num_dates+1)])
  differences_df = pd.DataFrame(indice_df.diff(axis = 1))
  differences_df.columns = ['difference_'+indice_name +'_'+ str(i) for i in range(num_dates)]
  indice_df['cat_' + indice_name] = cat_indices(np.nan_to_num(np.array(holder).T, nan=10, posinf=10, neginf=10)) 
  indice_df = pd.merge(indice_df, differences_df, how = 'left', left_index=True, right_index=True)
  return indice_df

In [ ]:
# A list of all indices
indices = [(aci, 'aci'), (avi, 'avi'), (bai, 'bai'), (bgi, 'bgi'), (bri, 'bri'), (ccci, 'ccci'), (cm, 'cm'), (dyi, 'dyi'), (evi, 'evi'), (exg, 'exg'),\
           (fidet, 'fidet'), (mcari, 'mcari'), (mi, 'mi'), (mrendvi, 'mrendvi'), (mresr, 'mresr'), (mtci, 'mtci'), (mtvi, 'mtvi'), (nbr, 'nbr'), \
           (ndsi, 'ndsi'), (nli, 'nli'), (ndmi, 'ndmi'), (pvi, 'pvi'), (s2rep, 's2rep'), (si, 'si'), (sipi, 'sipi'), (sr3, 'sr3'), (tcari, 'tcari'),\
           (tvi, 'tvi'), (vdvi, 'vdvi')]

# Calculating theindices
indices_dfs = []
for indice in indices:indices_dfs.append(indice_calculator(indice[0], indice[1]))  

# Merging the indices dataframes
indices_df = reduce(lambda  left,right: pd.merge(left,right,right_index=True, left_index=True,how='outer'), indices_dfs)
indices_df.head()

,aci_date_1,aci_date_2,aci_date_3,aci_date_4,aci_date_5,aci_date_6,aci_date_7,aci_date_8,aci_date_9,aci_date_10,aci_date_11,aci_date_12,aci_date_13,aci_date_14,aci_date_15,aci_date_16,aci_date_17,aci_date_18,aci_date_19,aci_date_20,aci_date_21,aci_date_22,aci_date_23,aci_date_24,aci_date_25,cat_aci,difference_aci_0,difference_aci_1,difference_aci_2,difference_aci_3,difference_aci_4,difference_aci_5,difference_aci_6,difference_aci_7,difference_aci_8,difference_aci_9,difference_aci_10,difference_aci_11,difference_aci_12,difference_aci_13,...,vdvi_date_12,vdvi_date_13,vdvi_date_14,vdvi_date_15,vdvi_date_16,vdvi_date_17,vdvi_date_18,vdvi_date_19,vdvi_date_20,vdvi_date_21,vdvi_date_22,vdvi_date_23,vdvi_date_24,vdvi_date_25,cat_vdvi,difference_vdvi_0,difference_vdvi_1,difference_vdvi_2,difference_vdvi_3,difference_vdvi_4,difference_vdvi_5,difference_vdvi_6,difference_vdvi_7,difference_vdvi_8,difference_vdvi_9,difference_vdvi_10,difference_vdvi_11,difference_vdvi_12,difference_vdvi_13,difference_vdvi_14,difference_vdvi_15,difference_vdvi_16,difference_vdvi_17,difference_vdvi_18,difference_vdvi_19,difference_vdvi_20,difference_vdvi_21,difference_vdvi_22,difference_vdvi_23,difference_vdvi_24
0,3964.794578,3092.632459,2842.374486,2867.459756,13000.607601,2355.120808,2525.288743,127462.005000,1311.120382,44529.070573,1291.538973,119495.445628,2367.499717,2264.476560,2050.866440,102427.727319,981.262859,3383.259998,5895.336977,70149.838093,3417.258015,4246.969951,4307.693779,4527.267934,4369.422295,0,NaN,-872.162119,-250.257973,25.085269,10133.147845,-10645.486793,170.167936,124936.716257,-126150.884618,43217.950191,-43237.531600,118203.906655,-117127.945912,-103.023157,...,0.001154,0.224321,0.282540,0.386902,-0.001576,0.115449,0.192095,0.042328,-0.004073,0.056985,0.024940,0.019448,0.011178,0.026680,0,NaN,0.000948,-0.001668,-0.004992,0.012629,-0.011713,-0.003285,0.000559,0.046777,-0.052970,0.248528,-0.241182,0.223168,0.058219,0.104362,-0.388478,0.117025,0.076646,-0.149768,-0.046401,0.061058,-0.032045,-0.005491,-0.008270,0.015501
1,13221.678692,5719.782079,5829.195700,9748.660066,24822.846333,3330.887451,3498.101034,114084.560812,2338.381497,20889.579047,2129.155159,15835.866628,2254.385630,2552.336030,2541.667947,59487.560428,35117.007086,3033.915298,4122.006014,40299.588759,2938.443553,3307.270561,4963.533955,4743.276397,5015.016070,1,NaN,-7501.896612,109.413621,3919.464366,15074.186267,-21491.958882,167.213583,110586.459779,-111746.179315,18551.197550,-18760.423888,13706.711469,-13581.480998,297.950400,...,0.031616,0.197324,0.194512,0.304348,-0.006213,0.000363,0.113456,0.017512,-0.004134,-0.033771,-0.032929,-0.037513,-0.043175,-0.030224,0,NaN,-0.000262,0.009591,0.013555,-0.005551,-0.009498,-0.002173,0.032317,-0.001682,-0.015838,0.168784,-0.132435,0.165708,-0.002813,0.109836,-0.310560,0.006576,0.113093,-0.095945,-0.021645,-0.029637,0.000842,-0.004584,-0.005663,0.012951
2,10108.172467,3611.055177,4258.556765,10121.363167,17001.101711,4067.038805,4260.948013,102005.208676,2604.009902,38998.446800,2550.748885,41537.781653,2400.314196,2331.545198,1247.327175,127453.652734,1560.089125,2354.311892,3482.794779,4828.214508,2727.828806,4505.371332,11242.312316,11866.274148,11378.326285,0,NaN,-6497.117289,647.501587,5862.806403,6879.738543,-12934.062905,193.909207,97744.260663,-99401.198774,36394.436898,-36447.697915,38987.032768,-39137.467457,-68.768998,...,0.004394,0.348037,0.365916,0.545921,0.000033,0.139968,0.272516,0.097637,0.017954,0.010591,-0.035018,-0.028345,-0.029930,-0.014897,0,NaN,-0.011636,0.003534,0.020581,-0.004546,-0.002735,0.005198,0.010719,0.063885,-0.066474,0.271566,-0.267735,0.343643,0.017880,0.180004,-0.545888,0.139935,0.132548,-0.174879,-0.079683,-0.007363,-0.045609,0.006673,-0.001585,0.015033
3,6245.544862,8695.338061,7112.943206,3847.333483,11404.801278,5481.647374,8375.711465,102015.408813,3694.255154,95314.497180,3270.220891,83176.321946,4822.873209,4759.621446,2235.885439,59977.947923,2588.473975,5172.074873,7526.253710,2

### Handling Categorical Features

In [ ]:
# Transform categorical features into the appropriate type
categorical_features = [x for x in indices_df.columns if 'cat' in x]
for feature in categorical_features: indices_df[feature] = indices_df[feature].astype('category')
indices_df[[x for x in indices_df.columns if 'cat' in x]].dtypes

cat_aci        category
cat_avi        category
cat_bai        category
cat_bgi        category
cat_bri        category
cat_ccci       category
cat_cm         category
cat_dyi        category
cat_evi        category
cat_exg        category
cat_fidet      category
cat_mcari      category
cat_mi         category
cat_mrendvi    category
cat_mresr      category
cat_mtci       category
cat_mtvi       category
cat_nbr        category
cat_ndsi       category
cat_nli        category
cat_ndmi       category
cat_pvi        category
cat_s2rep      category
cat_si         category
cat_sipi       category
cat_sr3        category
cat_tcari      category
cat_tvi        category
cat_vdvi       category
dtype: object

### Separating training and test sets

In [ ]:
train_indices = indices_df.loc[train.index]
test_indices= indices_df.loc[test.index]
train_indices.head()

,aci_date_1,aci_date_2,aci_date_3,aci_date_4,aci_date_5,aci_date_6,aci_date_7,aci_date_8,aci_date_9,aci_date_10,aci_date_11,aci_date_12,aci_date_13,aci_date_14,aci_date_15,aci_date_16,aci_date_17,aci_date_18,aci_date_19,aci_date_20,aci_date_21,aci_date_22,aci_date_23,aci_date_24,aci_date_25,cat_aci,difference_aci_0,difference_aci_1,difference_aci_2,difference_aci_3,difference_aci_4,difference_aci_5,difference_aci_6,difference_aci_7,difference_aci_8,difference_aci_9,difference_aci_10,difference_aci_11,difference_aci_12,difference_aci_13,...,vdvi_date_12,vdvi_date_13,vdvi_date_14,vdvi_date_15,vdvi_date_16,vdvi_date_17,vdvi_date_18,vdvi_date_19,vdvi_date_20,vdvi_date_21,vdvi_date_22,vdvi_date_23,vdvi_date_24,vdvi_date_25,cat_vdvi,difference_vdvi_0,difference_vdvi_1,difference_vdvi_2,difference_vdvi_3,difference_vdvi_4,difference_vdvi_5,difference_vdvi_6,difference_vdvi_7,difference_vdvi_8,difference_vdvi_9,difference_vdvi_10,difference_vdvi_11,difference_vdvi_12,difference_vdvi_13,difference_vdvi_14,difference_vdvi_15,difference_vdvi_16,difference_vdvi_17,difference_vdvi_18,difference_vdvi_19,difference_vdvi_20,difference_vdvi_21,difference_vdvi_22,difference_vdvi_23,difference_vdvi_24
0,3964.794578,3092.632459,2842.374486,2867.459756,13000.607601,2355.120808,2525.288743,127462.005000,1311.120382,44529.070573,1291.538973,119495.445628,2367.499717,2264.476560,2050.866440,102427.727319,981.262859,3383.259998,5895.336977,70149.838093,3417.258015,4246.969951,4307.693779,4527.267934,4369.422295,0,NaN,-872.162119,-250.257973,25.085269,10133.147845,-10645.486793,170.167936,124936.716257,-126150.884618,43217.950191,-43237.531600,118203.906655,-117127.945912,-103.023157,...,0.001154,0.224321,0.282540,0.386902,-0.001576,0.115449,0.192095,0.042328,-0.004073,0.056985,0.024940,0.019448,0.011178,0.026680,0,NaN,0.000948,-0.001668,-0.004992,0.012629,-0.011713,-0.003285,0.000559,0.046777,-0.052970,0.248528,-0.241182,0.223168,0.058219,0.104362,-0.388478,0.117025,0.076646,-0.149768,-0.046401,0.061058,-0.032045,-0.005491,-0.008270,0.015501
1,13221.678692,5719.782079,5829.195700,9748.660066,24822.846333,3330.887451,3498.101034,114084.560812,2338.381497,20889.579047,2129.155159,15835.866628,2254.385630,2552.336030,2541.667947,59487.560428,35117.007086,3033.915298,4122.006014,40299.588759,2938.443553,3307.270561,4963.533955,4743.276397,5015.016070,1,NaN,-7501.896612,109.413621,3919.464366,15074.186267,-21491.958882,167.213583,110586.459779,-111746.179315,18551.197550,-18760.423888,13706.711469,-13581.480998,297.950400,...,0.031616,0.197324,0.194512,0.304348,-0.006213,0.000363,0.113456,0.017512,-0.004134,-0.033771,-0.032929,-0.037513,-0.043175,-0.030224,0,NaN,-0.000262,0.009591,0.013555,-0.005551,-0.009498,-0.002173,0.032317,-0.001682,-0.015838,0.168784,-0.132435,0.165708,-0.002813,0.109836,-0.310560,0.006576,0.113093,-0.095945,-0.021645,-0.029637,0.000842,-0.004584,-0.005663,0.012951
2,10108.172467,3611.055177,4258.556765,10121.363167,17001.101711,4067.038805,4260.948013,102005.208676,2604.009902,38998.446800,2550.748885,41537.781653,2400.314196,2331.545198,1247.327175,127453.652734,1560.089125,2354.311892,3482.794779,4828.214508,2727.828806,4505.371332,11242.312316,11866.274148,11378.326285,0,NaN,-6497.117289,647.501587,5862.806403,6879.738543,-12934.062905,193.909207,97744.260663,-99401.198774,36394.436898,-36447.697915,38987.032768,-39137.467457,-68.768998,...,0.004394,0.348037,0.365916,0.545921,0.000033,0.139968,0.272516,0.097637,0.017954,0.010591,-0.035018,-0.028345,-0.029930,-0.014897,0,NaN,-0.011636,0.003534,0.020581,-0.004546,-0.002735,0.005198,0.010719,0.063885,-0.066474,0.271566,-0.267735,0.343643,0.017880,0.180004,-0.545888,0.139935,0.132548,-0.174879,-0.079683,-0.007363,-0.045609,0.006673,-0.001585,0.015033
3,6245.544862,8695.338061,7112.943206,3847.333483,11404.801278,5481.647374,8375.711465,102015.408813,3694.255154,95314.497180,3270.220891,83176.321946,4822.873209,4759.621446,2235.885439,59977.947923,2588.473975,5172.074873,7526.253710,2

### Model training - Indices

In [ ]:
# Model parameters
lgbm_params = {
    'n_estimators': 50000,
    'learning_rate': 0.05,
    'colsample_bytree': 0.2,
    'subsample': 0.2,
    'reg_alpha': 10,
    'reg_lambda': 5,
    'num_leaves': 20,
    'early_stopping_rounds': 300,
    'n_jobs': -1,
    'objective': 'multiclass',
    'boosting': 'gbdt',
    'feature_name': 'auto',
    'categorical_features': 'auto'
}

# Training lgbm model
indices_oof_predictions, loss = [], []
for train_index, test_index in StratifiedKFold(n_splits=10).split(train_indices, target):
  X_train, X_test, y_train, y_test = train_indices.iloc[train_index], train_indices.iloc[test_index], target[train_index], target[test_index]

  lgbm = LGBMClassifier(**lgbm_params)
  lgbm.fit(X_train, y_train, eval_set = [(X_test, y_test)], verbose = False)

  indices_oof_predictions.append(lgbm.predict_proba(test_indices))
  loss.append(log_loss(y_test, lgbm.predict_proba(X_test)))
  print('Loss: ', log_loss(y_test, lgbm.predict_proba(X_test)))

print('Indices Cross Validation Loss: ', np.mean(loss))

Loss:  0.5834403772828316
Loss:  0.6018028414877727
Loss:  0.6013919483549625
Loss:  0.5725120111449334
Loss:  0.5857960997761713
Loss:  0.5889456157105445
Loss:  0.593295276793316
Loss:  0.5903724833468865
Loss:  0.5880075664655969
Loss:  0.5900769135507116
Indices Cross Validation Loss:  0.5895641133913727


### Submission file preparation

In [ ]:
# Submission file preparation
submission_file = pd.DataFrame({'Field ID': test['Field ID']})
for i, j in enumerate(sample_submission.columns[1:]):
  submission_file[j] = np.mean(indices_oof_predictions, 0)[:, i]

submission_file.to_csv('LGBM_SUB.csv', index = False)
submission_file.head()

,Field ID,Crop_Lucerne/Medics,Crop_Planted pastures (perennial),Crop_Fallow,Crop_Wine grapes,Crop_Weeds,Crop_Small grain grazing,Crop_Wheat,Crop_Canola,Crop_Rooibos
87113,5,0.000168,0.001388,0.000289,0.000202,0.000022,0.013983,0.983030,0.000914,0.000005
87114,10,0.121958,0.662767,0.007526,0.001937,0.009663,0.187915,0.005728,0.001453,0.001055
87115,11,0.201377,0.233013,0.080443,0.000895,0.020700,0.427167,0.022013,0.012085,0.002308
87116,17,0.003301,0.023033,0.201360,0.000883,0.560976,0.121827,0.003304,0.000294,0.085022
87117,18,0.000211,0.042321,0.044460,0.002432,0.907361,0.002360,0.000271,0.000335,0.000248
